<a href="https://colab.research.google.com/github/2023284/Assignment1/blob/main/Interactive_Student_Dashboard_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

# For Jupyter Notebook
# %matplotlib widget

%matplotlib inline


In [42]:
df=pd.read_csv('students_dataset.csv')
df.head()

,Student,Programme,Gender,Module,Score,Attendance,StudyHours,Semester
0,Alice,BSc IT,Female,Python,58.9,87.6,8.8,Spring 2025
1,Alice,BSc IT,Female,Databases,80.1,80.4,6.4,Spring 2025
2,Alice,BSc IT,Female,Networks,64.3,77.6,0.2,Spring 2025
3,Alice,BSc IT,Female,Machine Learning,79.5,91.5,3.4,Spring 2025
4,Alice,BSc IT,Female,Statistics,65.7,79.1,5.9,Spring 2025


In [43]:
print(df.shape)
print(df.dtypes)
df.describe()


(50, 8)
Student        object
Programme      object
Gender         object
Module         object
Score         float64
Attendance    float64
StudyHours    float64
Semester       object
dtype: object


,Score,Attendance,StudyHours
count,50.000000,50.000000,50.000000
mean,70.706000,84.952000,7.032000
std,9.768053,6.956754,2.935319
min,48.600000,72.100000,0.200000
25%,64.625000,79.175000,5.075000
50%,70.100000,85.700000,7.100000
75%,77.500000,91.200000,8.775000
max,97.800000,99.800000,18.100000


In [48]:
Programme_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df["Programme"].unique().tolist()),
    value="All",
    description="Programme:",
    layout=widgets.Layout(width="250px")
)

Module_dropdown = widgets.Dropdown(
    options=["All"] + sorted(df["Module"].unique().tolist()),
    value="All",
    description="Module:",
    layout=widgets.Layout(width="250px")
)

min_score_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=5,
    description="Min Score:",
    continuous_update=False,
    layout=widgets.Layout(width="350px")
)


In [45]:
def filter_data(programme, module, min_score):
    filtered = df.copy()

    if programme != "All":
        filtered = filtered[filtered["Programme"] == programme]
    if module != "All":
        filtered = filtered[filtered["Module"] == module]

    filtered = filtered[filtered["Score"] >= min_score]
    return filtered


In [46]:
def update_dashboard(programme, module, min_score):
    data = filter_data(programme, module, min_score)

    if data.empty:
        print("No records match your filters.")
        return

    fig, axes = plt.subplots(2, 3, figsize=(16, 8))
    ax1, ax2, ax3 = axes[0]
    ax4, ax5, ax6 = axes[1]


    # [1] Avg Score by Module (bar)
    avg_score_module = data.groupby("Module")["Score"].mean().sort_values()
    ax1.bar(avg_score_module.index, avg_score_module.values)
    ax1.set_title("Average Score by Module")
    ax1.set_xlabel("Module")
    ax1.set_ylabel("Avg Score")
    ax1.set_ylim(0, 100)
    ax1.tick_params(axis='x', rotation=45)

    # [2] Score by Gender (boxplot)
    genders = data["Gender"].unique().tolist()
    box_data = [data[data["Gender"] == g]["Score"] for g in genders]
    ax2.boxplot(box_data, labels=genders, showmeans=True)
    ax2.set_title("Score Distribution by Gender")
    ax2.set_xlabel("Gender")
    ax2.set_ylabel("Score")
    ax2.set_ylim(0, 100)

    # [3] Score vs Attendance (scatter)
    ax3.scatter(data["Attendance"], data["Score"], alpha=0.7)
    ax3.set_title("Score vs Attendance")
    ax3.set_xlabel("Attendance (%)")
    ax3.set_ylabel("Score")
    ax3.set_xlim(40, 100)
    ax3.set_ylim(0, 100)

    # [4] Score Distribution (histogram)
    ax4.hist(data["Score"], bins=10, edgecolor="black", alpha=0.8)
    ax4.set_title("Score Distribution")
    ax4.set_xlabel("Score")
    ax4.set_ylabel("Frequency")
    ax4.set_xlim(0, 100)


    # [5] Avg Attendance by Module (bar)
    avg_att_module = data.groupby("Module")["Attendance"].mean().sort_values()
    ax5.bar(avg_att_module.index, avg_att_module.values)
    ax5.set_title("Average Attendance by Module")
    ax5.set_xlabel("Module")
    ax5.set_ylabel("Avg Attendance (%)")
    ax5.set_ylim(40, 100)
    ax5.tick_params(axis='x', rotation=45)

    # [6] Correlation Heatmap (Score, Attendance, StudyHours)
    numeric_cols = ["Score", "Attendance", "StudyHours"]
    corr = data[numeric_cols].corr()

    im = ax6.imshow(corr, vmin=-1, vmax=1)
    ax6.set_title("Correlation Heatmap")
    ax6.set_xticks(range(len(numeric_cols)))
    ax6.set_yticks(range(len(numeric_cols)))
    ax6.set_xticklabels(numeric_cols, rotation=45)
    ax6.set_yticklabels(numeric_cols)

    # Annotate correlation values
    for i in range(len(numeric_cols)):
        for j in range(len(numeric_cols)):
            ax6.text(j, i, f"{corr.iloc[i, j]:.2f}",
                     ha="center", va="center", color="white")

    fig.colorbar(im, ax=ax6, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()


In [47]:
controls = widgets.HBox([
    programme_dropdown,
    module_dropdown,
    min_score_slider
])

out = widgets.interactive_output(
    update_dashboard,
    {
        "programme": programme_dropdown,
        "module": module_dropdown,
        "min_score": min_score_slider
    }
)

display(controls, out)
